In [25]:
import datetime as dt
from bs4 import BeautifulSoup
import requests
import re
import time
import random
import pandas as pd
import os
import numpy as np
from itertools import permutations
from pulp import *

In [26]:
master_df = pd.read_csv('week_3.csv')

In [27]:
master_df

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,dk_pos_sal_avg_points
0,RB,Ezekiel Elliott,8300,DAL,25.45,20.7,21.050000
1,WR,DeAndre Hopkins,7900,ARI,26.45,21.5,18.078261
2,RB,Derrick Henry,7800,TEN,13.75,17.7,20.421053
3,RB,Dalvin Cook,7600,MIN,19.45,17.8,17.600000
4,WR,Julio Jones,7400,ATL,16.05,13.8,13.325000
...,...,...,...,...,...,...,...
404,DST,Lions,2400,DET,-1.00,5.9,6.831683
405,DST,Raiders,2300,LV,1.00,5.4,5.700935
406,DST,Panthers,2200,CAR,1.00,6.3,5.894737
407,DST,Texans,2100,HOU,2.00,5.4,4.621951


In [28]:
availables = master_df[['Pos','Name','Salary','Projected Points']]

In [29]:
availables = availables[["Pos", "Name", "Salary", "Projected Points"]].groupby(["Pos", "Name", "Salary", "Projected Points"]).agg("count")
availables = availables.reset_index()

In [30]:
availables

,Pos,Name,Salary,Projected Points
0,DST,49ers,3500,8.2
1,DST,Bears,2900,6.3
2,DST,Bengals,2600,6.0
3,DST,Bills,3400,6.2
4,DST,Broncos,3100,5.5
...,...,...,...,...
404,WR,Tyrie Cleveland,3000,1.4
405,WR,Ventell Bryant,3000,0.3
406,WR,Will Fuller V,6000,10.9
407,WR,Zach Pascal,4500,7.8


In [31]:
salaries = {}
points = {}
for pos in availables.Pos.unique():
    available_pos = availables[availables.Pos == pos]
    salary = list(available_pos[["Name","Salary"]].set_index("Name").to_dict().values())[0]
    point = list(available_pos[["Name","Projected Points"]].set_index("Name").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 4,
    "TE": 1,
    "FLEX": 1,
    "DST": 1,
}

pos_flex = {
    "QB": 0,
    "RB": 1,
    "WR": 1,
    "TE": 1,
    "FLEX": 0,
    "DST":0
}

pos_flex_available = 5

In [32]:
SALARY_CAP = 50000

In [33]:
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [34]:
for k, v in _vars.items():
    for i in v:
        print(lpSum([salaries[k][i] * _vars[k][i]]))
        print(lpSum([points[k][i] * _vars[k][i]]))
        print(lpSum([_vars[k][i]]))

3500*DST_49ers_
8.2*DST_49ers_
DST_49ers_
2900*DST_Bears_
6.3*DST_Bears_
DST_Bears_
2600*DST_Bengals_
6.0*DST_Bengals_
DST_Bengals_
3400*DST_Bills_
6.2*DST_Bills_
DST_Bills_
3100*DST_Broncos_
5.5*DST_Broncos_
DST_Broncos_
3800*DST_Browns_
7.8*DST_Browns_
DST_Browns_
3700*DST_Buccaneers_
8.3*DST_Buccaneers_
DST_Buccaneers_
3300*DST_Cardinals_
6.6*DST_Cardinals_
DST_Cardinals_
3600*DST_Chargers_
7.4*DST_Chargers_
DST_Chargers_
4100*DST_Colts_
8.4*DST_Colts_
DST_Colts_
2600*DST_Cowboys_
6.1*DST_Cowboys_
DST_Cowboys_
2800*DST_Eagles_
6.3*DST_Eagles_
DST_Eagles_
2800*DST_Falcons_
6.1*DST_Falcons_
DST_Falcons_
2700*DST_Giants_
6.7*DST_Giants_
DST_Giants_
2000*DST_Jets_
5.5*DST_Jets_
DST_Jets_
2400*DST_Lions_
5.9*DST_Lions_
DST_Lions_
2200*DST_Panthers_
6.3*DST_Panthers_
DST_Panthers_
3200*DST_Patriots_
7.0*DST_Patriots_
DST_Patriots_
2300*DST_Raiders_
5.4*DST_Raiders_
DST_Raiders_
3000*DST_Rams_
6.8*DST_Rams_
DST_Rams_
2900*DST_Seahawks_
5.8*DST_Seahawks_
DST_Seahawks_
4000*DST_Steelers_
9.0

In [35]:
pos_num_available['TE']

1

In [36]:
prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
position_constraints = []

# Setting up the reward
for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    prob += lpSum([pos_flex[k] * _vars[k][i] for i in v]) <= pos_flex_available
    
prob += lpSum(rewards)
prob += lpSum(costs) <= SALARY_CAP

In [37]:
position_constraints

[]

In [38]:
prob.solve()

1

In [39]:
def summary(prob):
    string = ''
    div = '---------------------------------------\n'
    string += "Variables:\n\n"
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue))
                       for const in constraints]
        if v.varValue != 0:
            string += v.name.replace('_',' ')
            string += " = "
            string += str(v.varValue)
            string += "\n"
    string += div
    string += "Constraints: "
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            string += "{} = {}".format(constraint_pretty, eval(constraint_pretty))
    string += "\n"
    string += div
    string += "Score: "
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    string += "{} = {}".format(score_pretty, eval(score))
    return string

In [40]:
stuff = summary(prob)

In [41]:
print(stuff)

Variables:

DST Panthers  = 1.0
QB Kyler Murray = 1.0
RB Devin Singletary = 1.0
RB Miles Sanders = 1.0
TE Jordan Reed = 1.0
WR Amari Cooper = 1.0
WR DeAndre Hopkins = 1.0
WR Diontae Johnson = 1.0
WR Terry McLaurin = 1.0
---------------------------------------
Constraints: 2200*1.0 + 6800*1.0 + 4900*1.0 + 6400*1.0 + 4000*1.0 + 6500*1.0 + 7900*1.0 + 5400*1.0 + 5900*1.0 = 50000.0
---------------------------------------
Score: 6.3*1.0 + 23.5*1.0 + 13.8*1.0 + 18.8*1.0 + 11.9*1.0 + 17.4*1.0 + 21.5*1.0 + 15.1*1.0 + 15.7*1.0 = 144.0
